In [23]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import pandas as pd

folder_path = "/Users/rohinpalsule/Documents/GitHub/GraphWalkFlights/Graph-Walk-Flight_Psiturk_by_Jerry_Rohin/data"

df_list = []

# Iterate through all CSV files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file)
        print(f"Reading: {file}")
        try:
            df_individual = pd.read_csv(file_path, on_bad_lines='skip')
            df_list.append(df_individual)
        except Exception as e:
            print(f"Error reading {file}: {e}")

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True)


Reading: S_Kirthin V. Rajkumar_2011_243.csv
Reading: final_S_ .csv


In [53]:
# Adding partid from worker id

df['partid'] = df['responses'].str.replace('{worker_id:',"").str.replace('}', '').str.replace("Kirthin V. Rajkumar","Kirthin") # for clarity
df.loc[df['partid'] == ' ', 'partid'] = 'Rayan' # Replacing id " " w his name
df['partid'] = df['partid'].ffill()

In [83]:
# Direct Memory df

direct_df = df[df['trial_type']=='directmemory_phase']

direct_df = direct_df.loc[:, ~direct_df.isna().all()]
direct_df = direct_df.drop(columns=['time_elapsed','cumulative_accuracy','weighted_accuracy','missedtrial']).reset_index(drop='index')
direct_df = direct_df.map(lambda x: x.replace('../static/images/US_Cities_List/', '') if isinstance(x, str) else x)

def get_chosen_city(row):
    if  row['key_press'] == 49:
        return row['stimulus_down_left']
    elif  row['key_press'] == 50:
        return row['stimulus_down_mid']
    elif  row['key_press'] == 51:
        return row['stimulus_down_right']
    else: return pd.NA

def get_wrong_distance(row):
    if pd.isna(row['stimulus_selected']):
        return pd.NA
    elif row['stimulus_selected'] == row['stimulus_short']:
        return 'close'
    elif row['stimulus_selected'] == row['stimulus_far']:
        return 'far'
    elif row['stimulus_selected'] == row['stimulus_correct']:
        return 'correct'
    else: return pd.NA

def get_weighted_correct(row):
    if pd.isna(row['wrong_distance']):
        return pd.NA
    elif row['wrong_distance'] == 'close':
        return 0.5
    elif row['wrong_distance'] == 'far':
        return 0
    elif row['wrong_distance'] == 'correct':
        return 1
    else: return pd.NA

direct_df['stimulus_selected'] = direct_df.apply(get_chosen_city,axis=1)
direct_df['wrong_distance'] = direct_df.apply(get_wrong_distance,axis=1)
direct_df['weighted_correct'] = direct_df.apply(get_weighted_correct,axis=1)
direct_df['weighted_correct'].mean(),direct_df['accuracy'].mean()

(np.float64(0.5714285714285714), np.float64(0.40625))